In [1]:
from sympy import *

# 1. $\dfrac{\partial{\textbf x^\top \textbf a}}{\partial{\textbf x}}=\textbf a$

In [2]:
m, n = symbols('m n')
x = MatrixSymbol('x', m, 1)
a = MatrixSymbol('a', m, 1)

In [3]:
expr = x.T * a
expr

x.T*a

In [4]:
d = diff(expr, x, evaluate = False)
d

Derivative(x.T*a, x)

In [5]:
d = d.doit()
d

a

得到了一个抽象的 $\textbf a$，而不是具体形式的向量

# 2.$\dfrac{\partial{\textbf a^\top \textbf X \textbf b}}{\partial{\textbf X}}=\textbf a \textbf b^\top$

矩阵的大小可以像上面一样写成符号

这里指定了具体的数，才能可视化打印出来

In [6]:
X = MatrixSymbol('X', 2, 3) # 'X', m, n
a = MatrixSymbol('a', 2, 1) # 'a', m, 1
b = MatrixSymbol('b', 3, 1) # 'X', n, 1

In [7]:
expr = a.T * X * b
expr

a.T*X*b

In [8]:
d = diff(expr, X, evaluate = False)
d

Derivative(a.T*X*b, X)

In [9]:
d = d.doit()
d

PermuteDims(ArrayTensorProduct(a, b), (3)(1 2))

这里的 `ArrayTensorProduct(a, b)` 就是 a 和 b 的 Tensor 乘法，得到一个 (2, 1, 3, 1) 的数组

相当于 $ab^\top$

然后使用 `PermuteDims` 将数组整理成 (2, 3, 1, 1)

In [10]:
d.as_explicit()

[[[[a[0, 0]*b[0, 0]]], [[a[0, 0]*b[1, 0]]], [[a[0, 0]*b[2, 0]]]], [[[a[1, 0]*b[0, 0]]], [[a[1, 0]*b[1, 0]]], [[a[1, 0]*b[2, 0]]]]]

用 `reshape` 转成二维矩阵

In [11]:
d.as_explicit().reshape(2,3)

[[a[0, 0]*b[0, 0], a[0, 0]*b[1, 0], a[0, 0]*b[2, 0]], [a[1, 0]*b[0, 0], a[1, 0]*b[1, 0], a[1, 0]*b[2, 0]]]

# 3.$\dfrac{\partial{\textbf a^\top \textbf X^\top \textbf b}}{\partial{\textbf X}}=\textbf b \textbf a^\top$

In [12]:
X = MatrixSymbol('X', 3, 2) # 'X', n, m
a = MatrixSymbol('a', 2, 1) # 'a', m, 1
b = MatrixSymbol('b', 3, 1) # 'X', n, 1

In [13]:
expr = a.T * X.T * b
expr

a.T*X.T*b

In [14]:
d = diff(expr, X, evaluate = False)
d

Derivative(a.T*X.T*b, X)

In [15]:
d = d.doit()
d

PermuteDims(ArrayTensorProduct(b, a), (1 2 3))

这里变成了 `ArrayTensorProduct(b, a)`

相当于 $ba^\top$

和上面的 `ArrayTensorProduct(a, b)` 不一样了

In [16]:
d.as_explicit()

[[[[a[0, 0]*b[0, 0]]], [[a[1, 0]*b[0, 0]]]], [[[a[0, 0]*b[1, 0]]], [[a[1, 0]*b[1, 0]]]], [[[a[0, 0]*b[2, 0]]], [[a[1, 0]*b[2, 0]]]]]

用 `reshape` 转成二维矩阵

In [17]:
d.as_explicit().reshape(3,2)

[[a[0, 0]*b[0, 0], a[1, 0]*b[0, 0]], [a[0, 0]*b[1, 0], a[1, 0]*b[1, 0]], [a[0, 0]*b[2, 0], a[1, 0]*b[2, 0]]]